In [27]:
import graphlab

In [28]:
sales = graphlab.SFrame('kc_house_data.gl/')

In [29]:
import numpy as np

In [30]:
from math import sqrt

###Function(SFrame s, features[] f, feature target) {
    return matrix columns_features_plus_columnWithConstant
    return array values_of_output
###}

In [31]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features
    
    # select the columns of data_SFrame given by the features 
    # list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    
    # assign the column of data_sframe associated with the output 
    # to the SArray output_sarray
    output_sarray = data_sframe[output]
    
    # the following will convert the SArray into a numpy array by 
    # first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [32]:
(example_features, example_output) = get_numpy_data(sales, 
                                                    ['sqft_living'], 
                                                    'price') 
print example_features[0,:]
print example_output[0] 

[  1.00000000e+00   1.18000000e+03]
221900.0


##Make a function to predict output

###Example:
my_weights = np.array([1., 1.]) # the example weights<br/>
my_features = example_features[0,] # we'll use the first data point<br/>
predicted_value = np.dot(my_features, my_weights)<br/>
print predicted_value

In [33]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In [34]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point

In [35]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


###Feature derivative function

In [36]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(errors, feature)
    return(derivative)

In [37]:
(example_features, example_output) = get_numpy_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


##Implement gradient descent

In [53]:
from math import sqrt

In [58]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions = predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        errors = predictions - output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative = feature_derivative(errors, feature_matrix[:, i])
            # add the squared value of the derivative to the gradient magnitude (for assessing convergence)
            gradient_sum_squares += derivative * derivative
            # subtract the step size times the derivative from the current weight
            weights[i] -= step_size * derivative
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

#Run the analysis

In [59]:
train_data,test_data = sales.random_split(.8,seed=0)

In [60]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [61]:
my_sqft_weights = regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

In [62]:
print my_sqft_weights

[-46999.88716555    281.91211912]


#Quiz Question:
**What is the value of the weight for sqft_living -- the second element of ‘simple_weights’ (rounded to 1 decimal place)?**

The value of the weight for sqft_living: 281.91211912<br/>
The value of the weight for sqft_living from first analysis: 310.26


#####Use your newly estimated weights and your predict_output() function to compute the predictions on all the TEST data (you will need to create a numpy array of the test feature_matrix and test output first:

In [63]:
(test_simple_feature_matrix, test_output) = get_numpy_data(test_data, simple_features, my_output)

In [64]:
predictions_test = predict_output(test_simple_feature_matrix, my_sqft_weights)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 1 (round to nearest dollar)?**

In [65]:
print predictions_test[0]

356134.443171


In [75]:
errors1 = test_output - predictions_test
rss_test = (errors1 * errors1).sum()
print rss_test

2.75400047593e+14


##Run a multiple regression

In [76]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [77]:
model_weights = regression_gradient_descent(feature_matrix, 
                                            output, initial_weights,
                                            step_size, tolerance)

In [78]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, model_features, my_output)
predictions_from_latestModel = predict_output(test_feature_matrix, model_weights)

**Quiz Question: What is the predicted price for the 1st house in the TEST data set for model 2 (round to nearest dollar)?**

In [79]:
print predictions_from_latestModel[0]

366651.412037


In [80]:
print test_data[0]['price']

310000.0


In [81]:
errors2 = test_output - predictions_from_latestModel
rss_test_from_latestModel = (errors2 * errors2).sum()
print rss_test_from_latestModel

2.70263446465e+14


**Quiz Question: Which estimate was closer to the true price for the 1st house on the Test data set, model 1 or model 2?**<br/>model 2

**Quiz Question: Which model (1 or 2) has lowest RSS on all of the TEST data? **